# Cleaning unique user data

In [161]:
# importing necessary packages
import pandas as pd
import requests
from dotenv import load_dotenv
load_dotenv()
import os
import json
import flickrapi
import time
import numpy as np
import re
#import fuzzywuzzy
#from fuzzywuzzy import fuzz, process
#import pycountry

In [162]:
# read csv into df
df_user_data = pd.read_csv('data/unique_owner.csv')

/var/folders/n9/f309cdzj1vg84wrrq1h5nskm0000gn/T/ipykernel_19156/63229535.py:2: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  df_user_data = pd.read_csv('data/unique_owner.csv')


In [163]:
df_user_data.shape

(38801, 38)

In [164]:
# check df
df_user_data.head(10)

,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,stat,person_id,person_nsid,...,person_has_free_educational_resources,person_timezone_label,person_timezone_offset,person_timezone_timezone_id,person_timezone_timezone,person_pro_badge,person_expire,person_mbox_sha1sum__content,code,message
0,0,0.0,0.0,0.0,0.0,0.0,0.0,ok,197843899@N04,197843899@N04,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1.0,1.0,1.0,1.0,1.0,1.0,ok,26745338@N06,26745338@N06,...,0.0,Brasilia,-03:00,America/Sao_Paulo,20.0,NaN,NaN,NaN,NaN,NaN
2,2,2.0,2.0,2.0,2.0,2.0,2.0,ok,31331368@N00,31331368@N00,...,0.0,"Canberra, Melbourne, Sydney",+10:00,Australia/Canberra,68.0,standard,0.000000e+00,NaN,NaN,NaN
3,3,3.0,3.0,3.0,3.0,3.0,3.0,ok,122687277@N03,122687277@N03,...,0.0,NaN,NaN,NaN,NaN,standard,0.000000e+00,NaN,NaN,NaN
4,4,4.0,4.0,4.0,4.0,4.0,4.0,ok,196275842@N08,196275842@N08,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,5.0,5.0,5.0,5.0,5.0,5.0,ok,44842144@N03,44842144@N03,...,0.0,NaN,NaN,NaN,NaN,standard,0.000000e+00,NaN,NaN,NaN
6,6,6.0,6.0,6.0,6.0,6.0,6.0,ok,45008553@N04,45008553@N04,...,0.0,"Bangkok, Hanoi, Jakarta",+07:00,Asia/Bangkok,55.0,NaN,NaN,NaN,NaN,NaN
7,7,7.0,7.0,7.0,7.0,7.0,7.0,ok,158117692@N02,158117692@N02,...,0.0,NaN,NaN,NaN,NaN,standard,1.614069e+09,NaN,NaN,NaN
8,8,8.0,8.0,8.0,8.0,8.0,8.0,ok,118071797@N03,118071797@N03,...,0.0,NaN,NaN,NaN,NaN,standard,0.000000e+00,NaN,NaN,NaN
9,9,9.0,9.0,9.0,9.0,9.0,9.0,ok,169491036@N06,169491036@N06,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Drop unnecessary columns

In [165]:
# check value count for is deleted
print(df_user_data.person_is_deleted.value_counts())
print(df_user_data.person_is_deleted.isna().sum())
# because there is only 1 value (False) and 24 NaN I decided to drop the column

0.0    38777
Name: person_is_deleted, dtype: int64
24


In [166]:
# list of columns to drop
columns_to_drop = ['Unnamed: 0.6','Unnamed: 0.5','Unnamed: 0.4', 'Unnamed: 0.3',
       'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'stat','person_nsid','person_is_deleted',
       'person_iconserver','person_iconfarm','person_has_stats',
       'person_description__content',
       'person_photosurl__content', 'person_profileurl__content',
       'person_mobileurl__content','person_has_adfree', 'person_has_free_standard_shipping',
       'person_has_free_educational_resources','person_pro_badge','person_mbox_sha1sum__content','message','person_expire', 'code','person_timezone_timezone']

In [167]:
# check if its the right number
len(columns_to_drop)

26

In [168]:
# drop 26 columns
df_user_data_cleaned = df_user_data.drop(columns=columns_to_drop)

In [169]:
# check columns
df_user_data_cleaned.columns

Index(['person_id', 'person_ispro', 'person_path_alias',
       'person_username__content', 'person_realname__content',
       'person_location__content', 'person_photos_firstdatetaken__content',
       'person_photos_firstdate__content', 'person_photos_count__content',
       'person_timezone_label', 'person_timezone_offset',
       'person_timezone_timezone_id'],
      dtype='object')

## Alter column names

In [170]:
# replace 'person' with user and delete '_content'
df_user_data_cleaned.columns = df_user_data_cleaned.columns.str.replace('person','user').str.replace('__content','')


In [171]:
df_user_data_cleaned.columns

Index(['user_id', 'user_ispro', 'user_path_alias', 'user_username',
       'user_realname', 'user_location', 'user_photos_firstdatetaken',
       'user_photos_firstdate', 'user_photos_count', 'user_timezone_label',
       'user_timezone_offset', 'user_timezone_timezone_id'],
      dtype='object')

## delete NaN from user_location, bc it's our most important column

In [172]:
# clear NaN values from user_locations
df_user_data_cleaned = df_user_data_cleaned.dropna(subset='user_location').reset_index(drop=True)
#df_user_data_cleaned = df_user_data_cleaned.reset_index(drop=True)

In [173]:
# check how much is still there
df_user_data_cleaned.shape

(14469, 12)

In [174]:
df_user_data_cleaned.head()

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id
0,26745338@N06,0.0,danisonksen,dani.sonksen,Daniela Sönksen,"Melbourne, Australia",2005-05-09 10:56:54,1.212696e+09,850.0,Brasilia,-03:00,America/Sao_Paulo
1,31331368@N00,1.0,lonelyradio,lonely radio,andrew j. cosgriff,"Melbourne, Australia",2000-02-01 00:00:00,1.099811e+09,13381.0,"Canberra, Melbourne, Sydney",+10:00,Australia/Canberra
2,122687277@N03,1.0,philipmallis,philip.mallis,Philip Mallis,Melbourne,2007-09-21 04:37:37,1.397393e+09,10793.0,NaN,NaN,NaN
3,45008553@N04,0.0,reaksmeyyean,REAKSMEY Yean – GEORGE,REAKSMEY Yean – GEORGE,"Phnom Penh, Kingdom of Cambodia",2005-11-04 20:32:27,1.259290e+09,236.0,"Bangkok, Hanoi, Jakarta",+07:00,Asia/Bangkok
4,158117692@N02,1.0,NaN,Julie McLennan,Julie McLennan,"Melbourne, Australia",2018-01-08 22:23:51,1.515479e+09,2533.0,NaN,NaN,NaN


In [175]:
len(df_user_data_cleaned)

14469

In [176]:
# check percentage of leftover data - 37% left data
x = (len(df_user_data_cleaned)/len(df_user_data))*100
print(x)

37.29027602381382


## alter dtypes

In [177]:
# check dtypes
df_user_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14469 entries, 0 to 14468
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   user_id                     14469 non-null  object 
 1   user_ispro                  14469 non-null  float64
 2   user_path_alias             9890 non-null   object 
 3   user_username               14469 non-null  object 
 4   user_realname               12591 non-null  object 
 5   user_location               14469 non-null  object 
 6   user_photos_firstdatetaken  14468 non-null  object 
 7   user_photos_firstdate       14468 non-null  float64
 8   user_photos_count           14469 non-null  float64
 9   user_timezone_label         13283 non-null  object 
 10  user_timezone_offset        13283 non-null  object 
 11  user_timezone_timezone_id   13283 non-null  object 
dtypes: float64(3), object(9)
memory usage: 1.3+ MB


In [178]:
# user_id to string
df_user_data_cleaned.user_id=df_user_data_cleaned.user_id.astype('string')

In [179]:
df_user_data_cleaned.user_id

0         26745338@N06
1         31331368@N00
2        122687277@N03
3         45008553@N04
4        158117692@N02
             ...      
14464    131418876@N03
14465     42905557@N00
14466     16939187@N06
14467     97454045@N07
14468     35939355@N04
Name: user_id, Length: 14469, dtype: string

In [180]:
# user_ispro to bool
df_user_data_cleaned.user_ispro=df_user_data_cleaned.user_ispro.astype('bool')

In [181]:
df_user_data_cleaned[['user_timezone_offset','user_timezone_timezone_id','user_timezone_label']].sample(5)

,user_timezone_offset,user_timezone_timezone_id,user_timezone_label
1967,+02:00,Africa/Harare,"Harare, Pretoria"
2101,-06:00,CST6CDT,Central Time (US & Canada)
6564,+01:00,Europe/Amsterdam,"Amsterdam, Berlin, Bern, Rome, Stockholm, Vienna"
9367,-08:00,PST8PDT,Pacific Time (US & Canada); Tijuana
6925,+01:00,Europe/Amsterdam,"Amsterdam, Berlin, Bern, Rome, Stockholm, Vienna"


In [182]:
# user_location','user_realname','user_username','user_path_alias,... to string except user_id, bc it needs to be original to query the API
df_user_data_cleaned[['user_location','user_realname','user_username','user_path_alias','user_timezone_offset','user_timezone_timezone_id','user_timezone_label']] = df_user_data_cleaned[['user_location','user_realname','user_username','user_path_alias','user_timezone_offset','user_timezone_timezone_id','user_timezone_label']].astype('string')

In [183]:

df_user_data_cleaned = df_user_data_cleaned.apply(lambda x: x.str.lower() if x.dtype == "string" else x) 

In [184]:
df_user_data_cleaned.sample(19)

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id
6143,198562431@n08,False,<NA>,stmaryspethaven.au,st marys pet haven,"st marys, australia",2018-09-21 16:10:08,1.686214e+09,16.0,<NA>,<NA>,<NA>
6172,186453034@n02,False,abortionpills247,abortionpillrx247,abortionpillrx247 online pharmacy,united states,2020-01-14 02:34:25,1.578998e+09,4.0,pacific time (us & canada); tijuana,-08:00,pst8pdt
5144,55889617@n03,False,<NA>,form of therapy,jae y,"cerritos, usa",2010-09-03 19:28:26,1.289709e+09,5017.0,pacific time (us & canada); tijuana,-08:00,pst8pdt
11392,64239684@n05,False,fotograf,fotografdude,lindsay,"brisbane, australia",2005-06-17 00:01:57,1.308572e+09,969.0,brisbane,+10:00,australia/brisbane
8416,63016432@n06,False,mariposa-latina,mariposa-latina,<NA>,kiel,2007-06-09 16:56:05,1.306965e+09,290.0,pacific time (us & canada); tijuana,-08:00,pst8pdt
3930,195382880@n02,True,<NA>,mkd photo,mike,"varese, italia",2022-05-29 09:38:27,1.653842e+09,18.0,pacific time (us & canada); tijuana,-08:00,etc/gmt+8
8295,44504749@n07,True,<NA>,anthonyvc,anthony corpuz,los angeles,2008-03-15 15:01:19,1.392700e+09,13527.0,taipei,+08:00,asia/taipei
4037,188722555@n03,False,dadavidgc_rns,dadavidgc_,david gutiérrez cisneros,"quito, ecuador",2018-02-12 18:54:13,1.591154e+09,60.0,"bogota, lima, quito",-05:00,america/bogota
4850,14841234@n02,True,rachelpasch,justmakeit,rachel,"illinois, usa",2000-01-18 17:09:16,1.192081e+09,2330.0,central time (us & canada),-06:00,cst6cdt
11154,62942199@n08,True,<NA>,takashi h,<NA>,"osaka, japan",2009-01-07 13:31:10,1.305428e+09,3443.0,"osaka, sapporo, tokyo",+09:00,asia/tokyo


In [185]:
df_user_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14469 entries, 0 to 14468
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   user_id                     14469 non-null  string 
 1   user_ispro                  14469 non-null  bool   
 2   user_path_alias             9890 non-null   string 
 3   user_username               14469 non-null  string 
 4   user_realname               12591 non-null  string 
 5   user_location               14469 non-null  string 
 6   user_photos_firstdatetaken  14468 non-null  object 
 7   user_photos_firstdate       14468 non-null  float64
 8   user_photos_count           14469 non-null  float64
 9   user_timezone_label         13283 non-null  string 
 10  user_timezone_offset        13283 non-null  string 
 11  user_timezone_timezone_id   13283 non-null  string 
dtypes: bool(1), float64(2), object(1), string(8)
memory usage: 1.2+ MB


In [186]:
# user_photos_count to int
df_user_data_cleaned.user_photos_count = df_user_data_cleaned.user_photos_count.astype('int')

In [187]:
df_user_data_cleaned.user_photos_firstdate

0        1.212696e+09
1        1.099811e+09
2        1.397393e+09
3        1.259290e+09
4        1.515479e+09
             ...     
14464    1.424362e+09
14465    1.134207e+09
14466    1.197543e+09
14467    1.371249e+09
14468    1.251922e+09
Name: user_photos_firstdate, Length: 14469, dtype: float64

In [188]:
# user_firstdate unix to datetime
df_user_data_cleaned.user_photos_firstdate = pd.to_datetime(df_user_data_cleaned.user_photos_firstdate,unit='s',errors='coerce')

In [189]:
# check type
type(df_user_data_cleaned.user_photos_firstdatetaken)

pandas.core.series.Series

In [190]:
# alter to string
df_user_data_cleaned.user_photos_firstdatetaken = df_user_data_cleaned.user_photos_firstdatetaken.astype('string')

In [191]:
df_user_data_cleaned.user_photos_firstdatetaken[9:15]

9     1840-01-01 00:00:00
10    2000-01-01 00:00:00
11    0000-00-00 00:00:00
12    0000-00-00 00:00:00
13    1980-01-01 00:00:07
14    1985-03-15 00:00:00
Name: user_photos_firstdatetaken, dtype: string

In [192]:
df_user_data_cleaned.user_photos_firstdatetaken = pd.to_datetime(df_user_data_cleaned.user_photos_firstdatetaken, dayfirst=True,format='%Y-%m-%d %H:%M:%S',errors='coerce')

In [193]:
# because coerce replaces error values with NaT 
# firstdatetaken is mysql datetime that starts with 1000-00-00 00:00:00 we "lost" 787 values
df_user_data_cleaned.user_photos_firstdatetaken.count()

13681

In [194]:
df_user_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14469 entries, 0 to 14468
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   user_id                     14469 non-null  string        
 1   user_ispro                  14469 non-null  bool          
 2   user_path_alias             9890 non-null   string        
 3   user_username               14469 non-null  string        
 4   user_realname               12591 non-null  string        
 5   user_location               14469 non-null  string        
 6   user_photos_firstdatetaken  13681 non-null  datetime64[ns]
 7   user_photos_firstdate       14468 non-null  datetime64[ns]
 8   user_photos_count           14469 non-null  int64         
 9   user_timezone_label         13283 non-null  string        
 10  user_timezone_offset        13283 non-null  string        
 11  user_timezone_timezone_id   13283 non-null  string    

## check for duplicates  

In [195]:
df_user_data_cleaned.duplicated().value_counts()

False    14177
True       292
dtype: int64

In [196]:
# we find more duplicates for one column, otherwise it compares the row
df_user_data_cleaned.duplicated(subset='user_id').value_counts()

False    14172
True       297
dtype: int64

In [197]:
# look at duplicates
df_user_data_cleaned[df_user_data_cleaned.duplicated(keep=False)].sort_values(by='user_id')

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id
1630,100175587@n02,True,<NA>,cbelato (sometimes out),cidinha belato,"rio de janeiro, brasil",2012-06-30 15:51:49,2013-08-13 00:04:46,2230,brasilia,-03:00,america/sao_paulo
1927,100175587@n02,True,<NA>,cbelato (sometimes out),cidinha belato,"rio de janeiro, brasil",2012-06-30 15:51:49,2013-08-13 00:04:46,2230,brasilia,-03:00,america/sao_paulo
1701,100175587@n02,True,<NA>,cbelato (sometimes out),cidinha belato,"rio de janeiro, brasil",2012-06-30 15:51:49,2013-08-13 00:04:46,2230,brasilia,-03:00,america/sao_paulo
1653,100193876@n06,True,juliek1967,julie from wexford,<NA>,"wexford, ireland",2004-11-04 12:43:48,2013-08-12 11:10:59,3396,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london
1724,100193876@n06,True,juliek1967,julie from wexford,<NA>,"wexford, ireland",2004-11-04 12:43:48,2013-08-12 11:10:59,3396,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london
...,...,...,...,...,...,...,...,...,...,...,...,...
1780,98309604@n00,True,kleepet,kleepet,karen peterson,"jersey city, nj, usa",2001-05-27 12:52:05,2005-08-17 06:37:30,6097,eastern time (us & canada),-05:00,est5edt
1893,98372297@n03,True,tiphainerolland,tiphaine rolland,tiphaine rolland,"toronto, canada",2010-08-01 00:00:00,2013-07-05 22:03:00,19104,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london
2119,98372297@n03,True,tiphainerolland,tiphaine rolland,tiphaine rolland,"toronto, canada",2010-08-01 00:00:00,2013-07-05 22:03:00,19104,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london
1827,98406434@n00,True,anyhoo,anyhoo,<NA>,uk,1998-07-01 00:00:00,2004-09-16 17:31:20,25403,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london


## keep existing user id's in user_in_df for updates

In [198]:
# drop duplicates and store existing user ids to a list
user_in_df=[]
df_user_data_cleaned = df_user_data_cleaned.drop_duplicates(subset='user_id')

for i in df_user_data_cleaned.user_id.unique():
    user_in_df.append(i)

In [199]:
df_user_data_cleaned.shape

(14172, 12)

In [200]:
# check if everything was right
print(user_in_df)
print(len(user_in_df))
# check type
type(user_in_df)

['26745338@n06', '31331368@n00', '122687277@n03', '45008553@n04', '158117692@n02', '169491036@n06', '153546415@n07', '78379788@n06', '115521748@n08', '44340545@n05', '45676495@n05', '96660892@n07', '196626126@n08', '45629904@n03', '49143546@n06', '165034204@n08', '64586261@n02', '197734992@n06', '94635768@n04', '197493901@n06', '90250231@n05', '17671746@n04', '79152742@n05', '18478048@n00', '126912597@n07', '21225329@n08', '149623494@n02', '149960743@n05', '75710752@n04', '197381218@n08', '31833404@n02', '97105070@n00', '185616361@n04', '46017193@n00', '21924357@n04', '30707075@n08', '25538536@n02', '8920035@n02', '15086926@n08', '191736856@n08', '155729724@n07', '153638711@n04', '38835423@n02', '191493143@n05', '187113622@n05', '26581186@n05', '199098213@n02', '98142888@n03', '29742055@n00', '97697738@n00', '194619535@n06', '155127994@n04', '58273812@n08', '135840124@n04', '98336874@n04', '60047372@n00', '198281398@n05', '10167052@n06', '7702423@n04', '8755091@n07', '8836196@n06', '12

list

In [201]:
df_user_data_cleaned.sample(5)

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id
9831,23408956@n02,False,<NA>,césar zarallo,césar zarallo,"madrid, spain",2006-01-01 00:00:00,2008-02-02 23:22:15,152,"brussels, copenhagen, madrid, paris",+01:00,europe/brussels
13004,76691719@n07,False,bookinghotelin,booking hotel in,alex,"skopje, macedonia",2000-01-01 00:00:00,2012-02-21 19:18:02,137,international date line west,-12:00,etc/gmt+12
2800,58415659@n00,True,kitmasterbloke,kitmasterbloke,steve knight,"halstead, united kingdom",1976-01-01 00:00:00,2009-01-15 13:28:38,50269,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london
3885,142072863@n02,False,autisticreality,autistic reality,alec frazier,"washington, dc, united states of america",2010-05-06 11:33:36,2016-04-18 15:05:30,73332,eastern time (us & canada),-05:00,est5edt
11910,31496313@n00,True,merlijnhoek,merlijn hoek,<NA>,"amsterdam zuidoost, netherlands",2000-01-04 00:00:00,2006-02-18 11:55:16,11454,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam


## split user location into useful columns

In [202]:
# check format of user_location
df_user_data_cleaned.user_location

0                    melbourne, australia
1                    melbourne, australia
2                               melbourne
3        phnom penh, kingdom of cambodia 
4                    melbourne, australia
                       ...               
14464            alcalá de henares, spain
14465                       padova, italy
14466                    cagliari, italia
14467                     szeged, hungary
14468                       dinan, france
Name: user_location, Length: 14172, dtype: string

In [203]:
## conclusion timestamp: as I figured out flickr is creating timezone columns with arguments of location, if they fit in at least on 'city'and/or 'country' format
# check how many values for timezone_offset
df_user_data_cleaned.user_timezone_offset.isna().sum()

1177

In [204]:
# note there is a wrong TZ
df_user_data_cleaned.loc[7152]

user_id                                                    91292806@n00
user_ispro                                                        False
user_path_alias                                              buferanera
user_username                           giovanni picuti, già buferanera
user_realname                                           giovanni picuti
user_location                 foligno, italy (giovanni.picuti@alice.it)
user_photos_firstdatetaken                          1970-12-01 00:00:00
user_photos_firstdate                               2006-11-09 22:04:04
user_photos_count                                                  2213
user_timezone_label                        international date line west
user_timezone_offset                                             -12:00
user_timezone_timezone_id                                    etc/gmt+12
Name: 7152, dtype: object

In [205]:
df_temp = df_user_data_cleaned.user_location.str.split(', ',expand=True)

In [206]:
df_temp

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,melbourne,australia,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,melbourne,australia,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,melbourne,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,phnom penh,kingdom of cambodia,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,melbourne,australia,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14464,alcalá de henares,spain,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
14465,padova,italy,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
14466,cagliari,italia,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
14467,szeged,hungary,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [207]:
df_temp.columns

RangeIndex(start=0, stop=22, step=1)

In [208]:
# "rename" columns from range to names
df_temp.columns=['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21']

In [209]:
# merge new location columns to dataframe
df_user_data_cleaned=pd.merge(df_user_data_cleaned,df_temp,left_index=True, right_index=True)

In [210]:
df_user_data_cleaned.head()

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,...,12,13,14,15,16,17,18,19,20,21
0,26745338@n06,False,danisonksen,dani.sonksen,daniela sönksen,"melbourne, australia",2005-05-09 10:56:54,2008-06-05 20:03:04,850,brasilia,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,31331368@n00,True,lonelyradio,lonely radio,andrew j. cosgriff,"melbourne, australia",2000-02-01 00:00:00,2004-11-07 06:59:09,13381,"canberra, melbourne, sydney",...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,122687277@n03,True,philipmallis,philip.mallis,philip mallis,melbourne,2007-09-21 04:37:37,2014-04-13 12:48:29,10793,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,45008553@n04,False,reaksmeyyean,reaksmey yean – george,reaksmey yean – george,"phnom penh, kingdom of cambodia",2005-11-04 20:32:27,2009-11-27 02:43:31,236,"bangkok, hanoi, jakarta",...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,158117692@n02,True,<NA>,julie mclennan,julie mclennan,"melbourne, australia",2018-01-08 22:23:51,2018-01-09 06:23:51,2533,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [211]:
# only the first 3 columns seem to have reliable data, so I dropped the rest
df_user_data_cleaned = df_user_data_cleaned.drop(columns=['3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21'])

In [212]:
df_user_data_cleaned.columns

Index(['user_id', 'user_ispro', 'user_path_alias', 'user_username',
       'user_realname', 'user_location', 'user_photos_firstdatetaken',
       'user_photos_firstdate', 'user_photos_count', 'user_timezone_label',
       'user_timezone_offset', 'user_timezone_timezone_id', '0', '1', '2'],
      dtype='object')

In [213]:
df_user_data_cleaned.shape

(14172, 15)

In [214]:
df_user_data_cleaned.head()

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2
0,26745338@n06,False,danisonksen,dani.sonksen,daniela sönksen,"melbourne, australia",2005-05-09 10:56:54,2008-06-05 20:03:04,850,brasilia,-03:00,america/sao_paulo,melbourne,australia,<NA>
1,31331368@n00,True,lonelyradio,lonely radio,andrew j. cosgriff,"melbourne, australia",2000-02-01 00:00:00,2004-11-07 06:59:09,13381,"canberra, melbourne, sydney",+10:00,australia/canberra,melbourne,australia,<NA>
2,122687277@n03,True,philipmallis,philip.mallis,philip mallis,melbourne,2007-09-21 04:37:37,2014-04-13 12:48:29,10793,<NA>,<NA>,<NA>,melbourne,<NA>,<NA>
3,45008553@n04,False,reaksmeyyean,reaksmey yean – george,reaksmey yean – george,"phnom penh, kingdom of cambodia",2005-11-04 20:32:27,2009-11-27 02:43:31,236,"bangkok, hanoi, jakarta",+07:00,asia/bangkok,phnom penh,kingdom of cambodia,<NA>
4,158117692@n02,True,<NA>,julie mclennan,julie mclennan,"melbourne, australia",2018-01-08 22:23:51,2018-01-09 06:23:51,2533,<NA>,<NA>,<NA>,melbourne,australia,<NA>


In [215]:
# check if there is a timezone generated with only usa
df_user_data_cleaned.loc[6316]

user_id                                             197323723@n06
user_ispro                                                  False
user_path_alias                                              <NA>
user_username                                    portauthursmiles
user_realname                                  port authur smiles
user_location                                       united states
user_photos_firstdatetaken                    2022-12-31 01:20:20
user_photos_firstdate                         2022-12-31 09:20:46
user_photos_count                                              11
user_timezone_label           pacific time (us & canada); tijuana
user_timezone_offset                                       -08:00
user_timezone_timezone_id                                 pst8pdt
0                                                   united states
1                                                            <NA>
2                                                            <NA>
Name: 6316

## figuring out what happens in the location column

In [216]:
# change dt to list
# split without expand=true
df_split = df_user_data_cleaned.user_location.str.split(', ')
df_split.tolist()

[['melbourne', 'australia'],
 ['melbourne', 'australia'],
 ['melbourne'],
 ['phnom penh', 'kingdom of cambodia '],
 ['melbourne', 'australia'],
 ['barishal', 'bangladesh'],
 ['melbourne', 'australia'],
 ['australia'],
 ['singapore', 'singapore'],
 ['manchester', 'greater manchester'],
 ['colton', 'england'],
 ['edinburgh', 'scotland'],
 ['west palm beach', 'united states'],
 ['brasília', 'brasil'],
 ['brasilia', 'brazil'],
 ['brasil'],
 ['brasília', 'brasil'],
 ['brasília (df)', 'brasil'],
 ['berlin', 'deutschland'],
 ['california', 'united states'],
 ['berlin  rabat paris'],
 ['malakoff', 'france'],
 ['bath', 'england'],
 ['hallein', 'austria'],
 ['poland'],
 ['karlovac', 'croatia'],
 ['poznań', 'poland'],
 ['austria'],
 ['burgos', 'españa'],
 ['bratislava', 'slovakia'],
 ['žilina', 'slovakia'],
 ['zagreb', 'croatia'],
 ['greenville', 'nc', 'united states'],
 ['devon', 'uk'],
 ['oslo', 'norway'],
 ['united states of america'],
 ['illogan', 'united kingdom'],
 ['hull', 'england'],
 ['m

### decided to only keep the first 3 columns as they fit the format

In [217]:
three_list=[]
for x in df_split:
    if len(x)<=3:
        three_list.append(x)
print(len(three_list))

14088


In [218]:
rest_list=[]
for x in df_split:
    if len(x)>=4:
        rest_list.append(x)
print(len(rest_list))

84


In [219]:
one_list=[]
for x in df_split:
    if len(x)<=1:
        one_list.append(x)
print(len(one_list))

3391


In [220]:
len(df_user_data_cleaned.user_location)

14172

In [221]:
# 94% follow the format
(len(rest_list)/len(three_list))*100

0.596252129471891

## getting new df's country,city, state

In [222]:
df_world_countries = pd.read_csv('data/world.csv')

In [223]:
df_world_countries.head(2)

,id,alpha2,alpha3,ar,bg,cs,da,de,el,en,...,ro,ru,sk,sl,sr,sv,th,uk,zh,zh-tw
0,4,af,afg,أفغانستان,Афганистан,Afghánistán,Afghanistan,Afghanistan,Αφγανιστάν,Afghanistan,...,Afganistan,Афганистан,Afganistan,Afganistan,Avganistan,Afghanistan,อัฟกานิสถาน,Афганістан,阿富汗,阿富汗
1,248,ax,ala,جزر أولاند,Оландски острови,Alandy,Ålandsøerne,Åland,Ώλαντ,Åland Islands,...,Insulele Åland,Аландские острова,Alandy,Ålandski otoki,Olandska Ostrva,Åland,หมู่เกาะโอลันด์,Аландські острови,奥兰,奧蘭


In [224]:
df_world_countries.drop(columns='id')

,alpha2,alpha3,ar,bg,cs,da,de,el,en,eo,...,ro,ru,sk,sl,sr,sv,th,uk,zh,zh-tw
0,af,afg,أفغانستان,Афганистан,Afghánistán,Afghanistan,Afghanistan,Αφγανιστάν,Afghanistan,Afganio,...,Afganistan,Афганистан,Afganistan,Afganistan,Avganistan,Afghanistan,อัฟกานิสถาน,Афганістан,阿富汗,阿富汗
1,ax,ala,جزر أولاند,Оландски острови,Alandy,Ålandsøerne,Åland,Ώλαντ,Åland Islands,Alando,...,Insulele Åland,Аландские острова,Alandy,Ålandski otoki,Olandska Ostrva,Åland,หมู่เกาะโอลันด์,Аландські острови,奥兰,奧蘭
2,al,alb,ألبانيا,Албания,Albánie,Albanien,Albanien,Αλβανία,Albania,Albanio,...,Albania,Албания,Albánsko,Albanija,Albanija,Albanien,แอลเบเนีย,Албанія,阿尔巴尼亚,阿爾巴尼亞
3,dz,dza,الجزائر,Алжир,Alžírsko,Algeriet,Algerien,Αλγερία,Algeria,Alĝerio,...,Algeria,Алжир,Alžírsko,Alžirija,Alžir,Algeriet,แอลจีเรีย,Алжир,阿尔及利亚,阿爾及利亞
4,as,asm,ساموا الأمريكية,Американска Самоа,Americká Samoa,Amerikansk Samoa,Amerikanisch-Samoa,Αμερικανική Σαμόα,American Samoa,Usona Samoo,...,Samoa americană,Американское Самоа,Americká Samoa,Ameriška Samoa,Američka Samoa,Amerikanska Samoa,อเมริกันซามัว,Американське Самоа,美属萨摩亚,美屬薩摩亞
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,wf,wlf,واليس وفوتونا,Уолис и Футуна,Wallis a Futuna,Wallis og Futuna,Wallis und Futuna,Ουαλίς και Φουτουνά,Wallis and Futuna,Valiso kaj Futuno,...,Wallis și Futuna,Уоллис и Футуна,Wallis a Futuna,Wallis in Futuna,Valis i Futuna,Wallis- och Futunaöarna,วอลิสและฟูตูนา,Волліс і Футуна,瓦利斯和富图纳,瓦利斯和富圖那
245,eh,esh,الصحراء الغربية,Западна Сахара,Západní Sahara,Vestsahara,Westsahara,Δυτική Σαχάρα,Western Sahara,Okcidenta Saharo,...,Sahara Occidentală,САДР,Západná Sahara,Zahodna Sahara,Zapadna Sahara,Västsahara,เวสเทิร์นสะฮารา,Західна Сахара,西撒哈拉,西撒哈拉
246,ye,yem,اليمن,Йемен,Jemen,Yemen,Jemen,Υεμένη,Yemen,Jemeno,...,Yemen,Йемен,Jemen,Jemen,Jemen,Jemen,เยเมน,Ємен,也门,葉門
247,zm,zmb,زامبيا,Замбия,Zambie,Zambia,Sambia,Ζάμπια,Zambia,Zambio,...,Zambia,Замбия,Zambia,Zambija,Zambija,Zambia,แซมเบีย,Замбія,赞比亚,尚比亞


In [225]:
df_world_countries.columns

Index(['id', 'alpha2', 'alpha3', 'ar', 'bg', 'cs', 'da', 'de', 'el', 'en',
       'eo', 'es', 'et', 'eu', 'fi', 'fr', 'hr', 'hu', 'hy', 'it', 'ja', 'ko',
       'lt', 'nl', 'no', 'pl', 'pt', 'ro', 'ru', 'sk', 'sl', 'sr', 'sv', 'th',
       'uk', 'zh', 'zh-tw'],
      dtype='object')

In [226]:
#creating a second column en
df_world_countries['en2']= df_world_countries['en']

In [227]:
# getting countries for laguages flickr has on the website (en,de,es,fr,zh,ko,it,pt,ru)

# creating dictionaries for each language

In [228]:
dict_country_ar= df_world_countries.set_index('ar')['en'].to_dict()
dict_country_bg= df_world_countries.set_index('bg')['en'].to_dict()
dict_country_cs= df_world_countries.set_index('cs')['en'].to_dict()
dict_country_da= df_world_countries.set_index('da')['en'].to_dict()
dict_country_de= df_world_countries.set_index('de')['en'].to_dict()
dict_country_el= df_world_countries.set_index('el')['en'].to_dict()
dict_country_en= df_world_countries.set_index('en')['en2'].to_dict()
dict_country_eo= df_world_countries.set_index('eo')['en'].to_dict()
dict_country_es= df_world_countries.set_index('es')['en'].to_dict()
dict_country_et= df_world_countries.set_index('et')['en'].to_dict()
dict_country_eu= df_world_countries.set_index('eu')['en'].to_dict()
dict_country_fi= df_world_countries.set_index('fi')['en'].to_dict()
dict_country_fr= df_world_countries.set_index('fr')['en'].to_dict()
dict_country_hr= df_world_countries.set_index('hr')['en'].to_dict()
dict_country_hu= df_world_countries.set_index('hu')['en'].to_dict()
dict_country_hy= df_world_countries.set_index('hy')['en'].to_dict()
dict_country_it= df_world_countries.set_index('it')['en'].to_dict()
dict_country_ja= df_world_countries.set_index('ja')['en'].to_dict()
dict_country_ko= df_world_countries.set_index('ko')['en'].to_dict()
dict_country_lt= df_world_countries.set_index('lt')['en'].to_dict()
dict_country_nl= df_world_countries.set_index('nl')['en'].to_dict()
dict_country_no= df_world_countries.set_index('no')['en'].to_dict()
dict_country_pl= df_world_countries.set_index('pl')['en'].to_dict()
dict_country_pt= df_world_countries.set_index('pt')['en'].to_dict()
dict_country_ro= df_world_countries.set_index('ro')['en'].to_dict()
dict_country_ru= df_world_countries.set_index('ru')['en'].to_dict()
dict_country_sk= df_world_countries.set_index('sk')['en'].to_dict()
dict_country_sl= df_world_countries.set_index('sl')['en'].to_dict()
dict_country_sr= df_world_countries.set_index('sr')['en'].to_dict()
dict_country_sv= df_world_countries.set_index('sv')['en'].to_dict()
dict_country_th= df_world_countries.set_index('th')['en'].to_dict()
dict_country_uk= df_world_countries.set_index('uk')['en'].to_dict()
dict_country_zh= df_world_countries.set_index('zh')['en'].to_dict()
dict_country_zh_tw= df_world_countries.set_index('zh-tw')['en'].to_dict()
dict_country_alpha2= df_world_countries.set_index('alpha2')['en'].to_dict()
dict_country_alpha3= df_world_countries.set_index('alpha3')['en'].to_dict()


In [229]:
# add all dictionaries to a list
list_world_countries = [dict_country_de,dict_country_ar,dict_country_bg,dict_country_cs,dict_country_da,dict_country_el,dict_country_en,dict_country_eo,dict_country_es,dict_country_et,dict_country_eu,dict_country_fi,dict_country_fr,dict_country_hr,dict_country_hu,dict_country_hy,dict_country_ja,dict_country_lt,dict_country_nl,dict_country_no,dict_country_pl,dict_country_pt,dict_country_ro,dict_country_sk,dict_country_sl,dict_country_sr,dict_country_sv,dict_country_th,dict_country_uk,dict_country_zh_tw,dict_country_zh,dict_country_ko,dict_country_it,dict_country_ru,dict_country_alpha2,dict_country_alpha3]
len(list_world_countries)

36

In [230]:
df_world_countries[df_world_countries.alpha2 == 'cz']

,id,alpha2,alpha3,ar,bg,cs,da,de,el,en,...,ru,sk,sl,sr,sv,th,uk,zh,zh-tw,en2
59,203,cz,cze,جمهورية التشيك,Чехия,Česko,Tjekkiet,Tschechien,Τσεχία,Czechia,...,Чехия,Česko,Češka,Češka,Tjeckien,เช็กเกีย,Чехія,捷克,捷克,Czechia


In [231]:
list_world_countries


[{'Afghanistan': 'Afghanistan',
  'Åland': 'Åland Islands',
  'Albanien': 'Albania',
  'Algerien': 'Algeria',
  'Amerikanisch-Samoa': 'American Samoa',
  'Andorra': 'Andorra',
  'Angola': 'Angola',
  'Anguilla': 'Anguilla',
  'Antarktis (Sonderstatus durch Antarktisvertrag)': 'Antarctica',
  'Antigua und Barbuda': 'Antigua and Barbuda',
  'Argentinien': 'Argentina',
  'Armenien': 'Armenia',
  'Aruba': 'Aruba',
  'Australien': 'Australia',
  'Österreich': 'Austria',
  'Aserbaidschan': 'Azerbaijan',
  'Bahamas': 'Bahamas',
  'Bahrain': 'Bahrain',
  'Bangladesch': 'Bangladesh',
  'Barbados': 'Barbados',
  'Belarus': 'Belarus',
  'Belgien': 'Belgium',
  'Belize': 'Belize',
  'Benin': 'Benin',
  'Bermuda': 'Bermuda',
  'Bhutan': 'Bhutan',
  'Bolivien': 'Bolivia (Plurinational State of)',
  'Bonaire, Saba, Sint Eustatius': 'Bonaire, Sint Eustatius and Saba',
  'Bosnien und Herzegowina': 'Bosnia and Herzegovina',
  'Botswana': 'Botswana',
  'Bouvetinsel': 'Bouvet Island',
  'Brasilien': 'Braz

In [232]:
df_user_data_cleaned.sample(5)

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2
5609,129533504@n06,True,alexjourba,alexjourba,alexandre jourba,"-, -",1994-02-08 12:40:07,2014-12-05 21:38:05,52656,"moscow, st. petersburg, volgograd",+03:00,europe/moscow,-,-,<NA>
5633,25105323@n05,True,susanet,susanamule,susana mulé,"buenos aires, argentina",2004-04-21 09:58:24,2008-10-09 19:28:18,2329,pacific time (us & canada); tijuana,-08:00,pst8pdt,buenos aires,argentina,<NA>
8125,23870567@n02,False,zhishi,zhishidesign,<NA>,russia,2007-06-08 16:11:00,2008-02-17 15:00:58,585,"moscow, st. petersburg, volgograd",+03:00,europe/moscow,russia,<NA>,<NA>
8731,30029895@n03,False,pacohurtado,paco hurtado,paco hurtado,"málaga, españa",2005-02-20 19:19:18,2008-09-04 21:05:15,790,"brussels, copenhagen, madrid, paris",+01:00,europe/brussels,málaga,españa,<NA>
10062,63801162@n03,True,stuartcollyerphotography,mrstuy,stuart collyer,kent,2008-09-20 21:09:00,2011-06-12 22:30:26,1269,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,kent,<NA>,<NA>


In [233]:
#replacing NaN with zero, bc Na is the country code for Namibia
df_user_data_cleaned['0'] = df_user_data_cleaned['0'].replace(np.nan,'0')
df_user_data_cleaned['1'] = df_user_data_cleaned['1'].replace(np.nan,'0')
df_user_data_cleaned['2'] = df_user_data_cleaned['2'].replace(np.nan,'0')

In [234]:
# combining the list of dict to one dictionary
combined_dict = {key:value for d in list_world_countries for key, value in d.items()}

In [235]:
type(combined_dict)

dict

In [236]:
# lower keys and values with dict comprehension
combined_dict = {k.lower(): v.lower() for k, v in combined_dict.items()}


In [237]:
combined_dict

{'afghanistan': 'afghanistan',
 'åland': 'åland islands',
 'albanien': 'albania',
 'algerien': 'algeria',
 'amerikanisch-samoa': 'american samoa',
 'andorra': 'andorra',
 'angola': 'angola',
 'anguilla': 'anguilla',
 'antarktis (sonderstatus durch antarktisvertrag)': 'antarctica',
 'antigua und barbuda': 'antigua and barbuda',
 'argentinien': 'argentina',
 'armenien': 'armenia',
 'aruba': 'aruba',
 'australien': 'australia',
 'österreich': 'austria',
 'aserbaidschan': 'azerbaijan',
 'bahamas': 'bahamas',
 'bahrain': 'bahrain',
 'bangladesch': 'bangladesh',
 'barbados': 'barbados',
 'belarus': 'belarus',
 'belgien': 'belgium',
 'belize': 'belize',
 'benin': 'benin',
 'bermuda': 'bermuda',
 'bhutan': 'bhutan',
 'bolivien': 'bolivia (plurinational state of)',
 'bonaire, saba, sint eustatius': 'bonaire, sint eustatius and saba',
 'bosnien und herzegowina': 'bosnia and herzegovina',
 'botswana': 'botswana',
 'bouvetinsel': 'bouvet island',
 'brasilien': 'brazil',
 'britisches territorium im

In [238]:
# add edge cases to dict
combined_dict ['england'] = 'england'
combined_dict ['scotland'] = 'scotland'
combined_dict ['united states'] = 'united states of america'
combined_dict ['kingdom of cambodia'] = 'cambodia'
combined_dict ['uk'] = 'united kingdom'
combined_dict ['wales'] = 'wales'
combined_dict ['united kingdom'] = 'united kingdom'
combined_dict ['czech republic'] = 'Czechia'

In [239]:
# clean columns to find more matching values
df_user_data_cleaned['0'] = df_user_data_cleaned['0'].str.lower().str.strip()
df_user_data_cleaned['1'] = df_user_data_cleaned['1'].str.lower().str.strip()
df_user_data_cleaned['2'] = df_user_data_cleaned['2'].str.lower().str.strip()

In [240]:
# creating a new column empty column with NaN's
df_user_data_cleaned['user_country']= np.nan


In [241]:
df_user_data_cleaned.sample(5)

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2,user_country
3958,37072378@n08,True,pyride,orchids love rainwater,julie anne johnson,"cheltenham, uk",2008-05-12 10:58:38,2009-05-31 13:41:25,2495,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,cheltenham,uk,0,NaN
8997,144857366@n07,True,<NA>,anja anlauf,anja anlauf,dinslaken,2010-12-18 13:02:39,2016-07-08 17:34:39,402,pacific time (us & canada); tijuana,-08:00,pst8pdt,dinslaken,0,0,NaN
4451,193290786@n04,False,sgg_hobby,hobbymip,葉如豪,"akl, nzl",2019-03-08 20:37:20,2021-10-02 04:27:57,89,"auckland, wellington",+12:00,pacific/auckland,akl,nzl,0,NaN
6117,197372296@n05,False,<NA>,eduastro,eduardo suárez,united kingdom,2022-12-10 15:18:02,2023-01-10 19:18:23,81,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,united kingdom,0,0,NaN
6025,163365711@n04,True,<NA>,reinier de rooie,reinier de rooie,"culemborg, the netherlands",2007-02-27 14:43:38,2019-04-21 14:30:52,2501,<NA>,<NA>,<NA>,culemborg,the netherlands,0,NaN


## iterating vice versa, bc otherwise shortcuts for states will be recognized as countries

In [242]:
# iterating over every row and dict to find fitting countries
for index, row in df_user_data_cleaned.iterrows():
    zero = row['2']
    if pd.isna(row['user_country']):
        if zero in combined_dict:
            df_user_data_cleaned.at[index, 'user_country']= combined_dict[zero] #fuzzywuzzy .apply

In [243]:
# iterating over every row and dict to find fitting countries
for index, row in df_user_data_cleaned.iterrows():
    zero = row['1']
    if pd.isna(row['user_country']):
        if zero in combined_dict:
            df_user_data_cleaned.at[index, 'user_country']= combined_dict[zero] #fuzzywuzzy .apply

In [244]:
# iterating over every row and dict to find fitting countries
for index, row in df_user_data_cleaned.iterrows():
    zero = row['0']
    if pd.isna(row['user_country']):
        if zero in combined_dict:
            df_user_data_cleaned.at[index, 'user_country']= combined_dict[zero] #fuzzywuzzy .apply


In [245]:
df_user_data_cleaned.head(25)

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2,user_country
0,26745338@n06,False,danisonksen,dani.sonksen,daniela sönksen,"melbourne, australia",2005-05-09 10:56:54,2008-06-05 20:03:04,850,brasilia,-03:00,america/sao_paulo,melbourne,australia,0,australia
1,31331368@n00,True,lonelyradio,lonely radio,andrew j. cosgriff,"melbourne, australia",2000-02-01 00:00:00,2004-11-07 06:59:09,13381,"canberra, melbourne, sydney",+10:00,australia/canberra,melbourne,australia,0,australia
2,122687277@n03,True,philipmallis,philip.mallis,philip mallis,melbourne,2007-09-21 04:37:37,2014-04-13 12:48:29,10793,<NA>,<NA>,<NA>,melbourne,0,0,NaN
3,45008553@n04,False,reaksmeyyean,reaksmey yean – george,reaksmey yean – george,"phnom penh, kingdom of cambodia",2005-11-04 20:32:27,2009-11-27 02:43:31,236,"bangkok, hanoi, jakarta",+07:00,asia/bangkok,phnom penh,kingdom of cambodia,0,cambodia
4,158117692@n02,True,<NA>,julie mclennan,julie mclennan,"melbourne, australia",2018-01-08 22:23:51,2018-01-09 06:23:51,2533,<NA>,<NA>,<NA>,melbourne,australia,0,australia
5,169491036@n06,False,<NA>,sanzidrahmankhantamim,sanzid rahman khan tamim,"barishal, bangladesh",2015-03-24 23:04:05,2019-04-06 13:35:11,407,<NA>,<NA>,<NA>,barishal,bangladesh,0,bangladesh
6,153546415@n07,True,free_aza_bird,free_aza_bird,karl baker,"melbourne, australia",2015-06-22 08:50:59,2017-04-03 01:24:16,2674,"canberra, melbourne, sydney",+10:00,australia/canberra,melbourne,australia,0,australia
7,78379788@n06,True,<NA>,rob lee photography,rob lee,australia,1985-01-01 00:00:00,2014-08-20 06:35:22,2867,"canberra, melbourne, sydney",+10:00,australia/canberra,australia,0,0,australia
8,115521748@n08,False,<NA>,footeefok,tee fok foo,"singapore, singapore",2008-08-24 12:38:45,2014-01-25 15:18:44,20678,"kuala lumpur, singapore",+08:00,asia/kuala_lumpur,singapore,singapore,0,singapore
9,44340545@n05,True,gmpolice1,greater manchester police,greater manchester police,"manchester, greater manchester",1840-01-01 00:00:00,2009-11-05 13:53:07,27764,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,manchester,greater manchester,0,NaN


In [246]:
combined_dict['kingdom of cambodia']

'cambodia'

### fuzzywuzzy function

In [247]:
# def fuzzy_match_country(text, choices):
#     # Use fuzzywuzzy to find the best match
#     best_match, score = process.extractOne(text, choices, scorer=fuzz.ratio)
#     # Set a threshold for a minimum score to consider a match
#     threshold = 80  # You can adjust this threshold as needed
#     if score >= threshold:
#         return best_match
#     else:
#         return None

In [248]:
# Clean up the 'country_code_name' column
#df_user_data_cleaned['user_country_2'] = df_user_data_cleaned['1'].apply(lambda x: fuzzy_match_country(x, combined_dict.keys()))

In [249]:
#df_user_data_cleaned['user_country_2'].value_counts()

In [250]:
df_user_data_cleaned.sample(50)

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2,user_country
10651,198145343@n06,False,<NA>,dr.amadaziz,ahmad aziz,"london, uk",2023-07-15 08:30:26,2023-07-15 15:30:34,11,<NA>,<NA>,<NA>,london,uk,0,united kingdom
7245,186591801@n08,True,<NA>,th.mueller,thomas müller,"rödental, germany",1984-06-01 00:00:00,2020-01-19 10:36:21,11953,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,rödental,germany,0,germany
1525,41824260@n08,True,<NA>,steve.schlick,steven schlichtmann,"brooklin, canada",2009-10-24 10:20:15,2009-10-25 23:23:40,1088,eastern time (us & canada),-05:00,est5edt,brooklin,canada,0,canada
13719,11683088@n00,False,kanatin,kanatin,kanatin,japan,2003-02-12 15:33:19,2006-04-25 10:42:52,20040,"osaka, sapporo, tokyo",+09:00,asia/tokyo,japan,0,0,japan
6580,194672020@n05,False,<NA>,enteriscloud,enteris cloud,"casper, united states",2013-07-27 16:05:01,2021-12-28 08:52:39,122,<NA>,<NA>,<NA>,casper,united states,0,united states of america
13357,88711965@n00,False,thebossfrombcp,manu arts,<NA>,"lima, perú",2003-01-01 00:00:00,2006-04-17 05:00:54,21530,pacific time (us & canada); tijuana,-08:00,pst8pdt,lima,perú,0,peru
11699,50596119@n06,False,beenleighmeadowsfarm,radiance yoga totnes,steph @ radiance yoga,"totnes, uk",2002-01-08 11:57:03,2010-05-26 20:49:42,405,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,totnes,uk,0,united kingdom
7392,138911778@n05,True,<NA>,marcel kochen,marcel kochen,"nieuwegein, netherlands",1938-01-01 00:00:00,2017-12-08 13:51:35,3716,pacific time (us & canada); tijuana,-08:00,pst8pdt,nieuwegein,netherlands,0,netherlands
103,184624976@n04,False,michael_tradovate,michael.tradovate,michael alford,bristol uk,2017-03-09 03:04:31,2019-10-04 19:45:45,471,<NA>,<NA>,<NA>,bristol uk,0,0,NaN
5288,117785599@n03,False,<NA>,lee reynolds1,lee reynolds,"neath, wales, united kingdon",2012-10-13 12:02:58,2014-02-26 20:41:08,819,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,neath,wales,united kingdon,wales


### check edge cases

In [251]:
df_user_data_cleaned.loc[12714]

user_id                                      91396833@n00
user_ispro                                           True
user_path_alias                                  jmd41280
user_username                                    jmd41280
user_realname                                  jon dawson
user_location                 north belle vernon, pa, usa
user_photos_firstdatetaken            1997-06-16 11:28:21
user_photos_firstdate                 2008-09-13 17:07:18
user_photos_count                                    9470
user_timezone_label            eastern time (us & canada)
user_timezone_offset                               -05:00
user_timezone_timezone_id                         est5edt
0                                      north belle vernon
1                                                      pa
2                                                     usa
user_country                     united states of america
Name: 12714, dtype: object

In [252]:
df_user_data_cleaned.loc[12247]

user_id                                                            96733159@n00
user_ispro                                                                 True
user_path_alias                                                     michalkovac
user_username                                                      michal kovac
user_realname                                                      michal kováč
user_location                                            prague, czech republic
user_photos_firstdatetaken                                  2006-07-26 10:42:34
user_photos_firstdate                                       2006-08-20 20:52:09
user_photos_count                                                          1634
user_timezone_label           belgrade, bratislava, budapest, ljubljana, prague
user_timezone_offset                                                     +01:00
user_timezone_timezone_id                                       europe/belgrade
0                                       

In [253]:
df_user_data_cleaned.loc[657]

user_id                                                           55189709@n06
user_ispro                                                                True
user_path_alias                                                       dvdhaven
user_username                                                     dvdhaven :-)
user_realname                                               dolf van der haven
user_location                                      groenekan, the netherlands 
user_photos_firstdatetaken                                                 NaT
user_photos_firstdate                                      2010-10-30 20:02:11
user_photos_count                                                         1336
user_timezone_label           amsterdam, berlin, bern, rome, stockholm, vienna
user_timezone_offset                                                    +01:00
user_timezone_timezone_id                                     europe/amsterdam
0                                                   

In [254]:
combined_dict['cz']

'czechia'

In [255]:
# changing czechia to czech republic
df_user_data_cleaned.user_country.replace('czechia','czech republic', inplace=True)

In [256]:
# df_user_data_cleaned['user_country_5'] = df_user_data_cleaned['1'].apply(lambda x: fuzzy_match_country(x, countries_dict_na_na.keys()))

## cities

In [257]:
df_world_cities = pd.read_csv('data/geonames-all-cities-with-a-population-1000.csv', delimiter=';')

In [258]:
df_world_cities.sample(8)

,Geoname ID,Name,ASCII Name,Alternate Names,Feature Class,Feature Code,Country Code,Country name EN,Country Code 2,Admin1 Code,Admin2 Code,Admin3 Code,Admin4 Code,Population,Elevation,DIgital Elevation Model,Timezone,Modification date,LABEL EN,Coordinates
126659,2777685,Großharras,Grossharras,NaN,P,PPLA3,AT,Austria,NaN,03,316,31616,NaN,482,NaN,195,Europe/Vienna,2018-07-28,Austria,"48.66389, 16.24556"
7417,2797641,Genval,Genval,Rixensart (Genval),P,PPL,BE,Belgium,NaN,WAL,WBR,25,25091,8253,NaN,101,Europe/Brussels,2020-05-25,Belgium,"50.72162, 4.49375"
1593,4008076,Testerazo,Testerazo,"El Refugio,Testerazo",P,PPL,MX,Mexico,NaN,18,008,NaN,NaN,2410,NaN,1029,America/Mazatlan,2018-11-03,Mexico,"21.40242, -104.8942"
67639,2127802,Tōbetsu,Tobetsu,"Tobetsu,Tōbetsu,dang bie ding,当別町",P,PPL,JP,Japan,NaN,12,7428085,01303,NaN,16694,NaN,14,Asia/Tokyo,2021-10-01,Japan,"43.21694, 141.51694"
92580,456506,Priekule,Priekule,"Preekuln,Prekule,Prekuln,Priekule,Priyekule,Pr...",P,PPLA2,LV,Latvia,NaN,DN,0641615,NaN,NaN,2550,NaN,59,Europe/Riga,2021-08-16,Latvia,"56.44679, 21.58968"
78774,1815196,Chengwu,Chengwu,"Ch'eng-wu-hsien,Chengwu,Ch’eng-wu-hsien,cheng ...",P,PPLA3,CN,China,NaN,25,3717,NaN,NaN,0,NaN,47,Asia/Shanghai,2021-09-20,China,"34.95407, 115.88419"
39851,863710,Sterche,Sterche,"Starcea,Sterche,Stârcea,Стерче",P,PPL,UA,Ukraine,NaN,03,7306,NaN,NaN,1231,349.0,351,Europe/Kyiv,2022-10-27,Ukraine,"48.0321, 26.03533"
2344,3169746,Pusiano,Pusiano,"Pasiano,Pusiano",P,PPLA3,IT,Italy,NaN,09,CO,013193,NaN,1327,264.0,278,Europe/Rome,2014-04-13,Italy,"45.81478, 9.28217"


In [259]:
# clean
df_world_cities.columns = df_world_cities.columns.str.lower().str.replace(' ', '_')


In [260]:
city_columns_to_keep = ['name','ascii_name', 'alternate_names']

In [261]:
df_world_cities_split = df_world_cities.loc[:,city_columns_to_keep]

In [262]:
df_world_cities_split

,name,ascii_name,alternate_names
0,Lyudinovo,Lyudinovo,"Ljudinovo,Lyudinovo,Людиново"
1,Lubyany,Lubyany,"Lubjany,Lubyany,Лубяны"
2,Lobanovo,Lobanovo,"Lobanovo,Лобаново"
3,Linda,Linda,"Linda,Линда"
4,Leskolovo,Leskolovo,"Leskolovo,Lieskula,Лесколово"
...,...,...,...
141116,Limeira,Limeira,"Limeira,Limejra,QGB,li mei la,li mey ra,limaye..."
141117,Itaúna,Itauna,"Itauna,ita'una,yi ta wu na,Итауна,Իտաունա,ইটাউ..."
141118,Itaporanga,Itaporanga,NaN
141119,Itapicuru,Itapicuru,"Missao,Missão"


In [263]:
df_world_cities_split.name = df_world_cities_split.name.str.strip().str.lower()
df_world_cities_split.ascii_name = df_world_cities_split.ascii_name.str.strip().str.lower()
df_world_cities_split.alternate_names = df_world_cities_split.alternate_names.str.strip().str.lower()

In [264]:
df_alternate_names = df_world_cities_split.alternate_names.str.split(',', expand=True)

In [265]:
df_alternate_names.sample(8)

,0,1,2,3,4,5,6,7,8,9,...,259,260,261,262,263,264,265,266,267,268
119124,san maurizio d'opaglio,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
93975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29745,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39973,grande ecaille,grande port,port sulphur,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9092,awbari,awbārī,qub,ubari,awbary,أوباري,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
89330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [266]:
df_alternate_names.columns = df_alternate_names.columns.astype('string')

In [267]:
columns_keep = ['0','1','2','3','4','5','6']

In [268]:
df_alternate_names = df_alternate_names.loc[:,columns_keep]

In [269]:
df_alternate_names

,0,1,2,3,4,5,6
0,ljudinovo,lyudinovo,людиново,None,None,None,None
1,lubjany,lubyany,лубяны,None,None,None,None
2,lobanovo,лобаново,None,None,None,None,None
3,linda,линда,None,None,None,None,None
4,leskolovo,lieskula,лесколово,None,None,None,None
...,...,...,...,...,...,...,...
141116,limeira,limejra,qgb,li mei la,li mey ra,limaye'ira,лимеира
141117,itauna,ita'una,yi ta wu na,итауна,իտաունա,ইটাউনা,伊塔乌纳
141118,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141119,missao,missão,None,None,None,None,None


In [270]:
# merge to split df

In [271]:
df_world_cities_split = pd.merge(df_world_cities_split,df_alternate_names, left_index=True,right_index=True)

In [272]:
df_world_cities_split

,name,ascii_name,alternate_names,0,1,2,3,4,5,6
0,lyudinovo,lyudinovo,"ljudinovo,lyudinovo,людиново",ljudinovo,lyudinovo,людиново,None,None,None,None
1,lubyany,lubyany,"lubjany,lubyany,лубяны",lubjany,lubyany,лубяны,None,None,None,None
2,lobanovo,lobanovo,"lobanovo,лобаново",lobanovo,лобаново,None,None,None,None,None
3,linda,linda,"linda,линда",linda,линда,None,None,None,None,None
4,leskolovo,leskolovo,"leskolovo,lieskula,лесколово",leskolovo,lieskula,лесколово,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
141116,limeira,limeira,"limeira,limejra,qgb,li mei la,li mey ra,limaye...",limeira,limejra,qgb,li mei la,li mey ra,limaye'ira,лимеира
141117,itaúna,itauna,"itauna,ita'una,yi ta wu na,итауна,իտաունա,ইটাউ...",itauna,ita'una,yi ta wu na,итауна,իտաունա,ইটাউনা,伊塔乌纳
141118,itaporanga,itaporanga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141119,itapicuru,itapicuru,"missao,missão",missao,missão,None,None,None,None,None


In [273]:
df_world_cities_split = df_world_cities_split.drop(columns='alternate_names')

In [274]:
df_world_cities_split['ascii_name_2'] = df_world_cities_split.ascii_name

In [275]:
df_world_cities_split

,name,ascii_name,0,1,2,3,4,5,6,ascii_name_2
0,lyudinovo,lyudinovo,ljudinovo,lyudinovo,людиново,None,None,None,None,lyudinovo
1,lubyany,lubyany,lubjany,lubyany,лубяны,None,None,None,None,lubyany
2,lobanovo,lobanovo,lobanovo,лобаново,None,None,None,None,None,lobanovo
3,linda,linda,linda,линда,None,None,None,None,None,linda
4,leskolovo,leskolovo,leskolovo,lieskula,лесколово,None,None,None,None,leskolovo
...,...,...,...,...,...,...,...,...,...,...
141116,limeira,limeira,limeira,limejra,qgb,li mei la,li mey ra,limaye'ira,лимеира,limeira
141117,itaúna,itauna,itauna,ita'una,yi ta wu na,итауна,իտաունա,ইটাউনা,伊塔乌纳,itauna
141118,itaporanga,itaporanga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,itaporanga
141119,itapicuru,itapicuru,missao,missão,None,None,None,None,None,itapicuru


In [276]:
dict_city_ascii_name= df_world_cities_split.set_index('ascii_name_2')['ascii_name'].to_dict()
dict_city_zero= df_world_cities_split.set_index('0')['ascii_name'].to_dict()
dict_city_one= df_world_cities_split.set_index('1')['ascii_name'].to_dict()
dict_city_two= df_world_cities_split.set_index('2')['ascii_name'].to_dict()
dict_city_three= df_world_cities_split.set_index('3')['ascii_name'].to_dict()
dict_city_four= df_world_cities_split.set_index('4')['ascii_name'].to_dict()
dict_city_five= df_world_cities_split.set_index('5')['ascii_name'].to_dict()
dict_city_six= df_world_cities_split.set_index('6')['ascii_name'].to_dict()

In [277]:
dict_city_name= df_world_cities_split.set_index('name')['ascii_name'].to_dict()

In [278]:
list_world_cities = [dict_city_zero,dict_city_ascii_name,dict_city_one,dict_city_two,dict_city_three,dict_city_four,dict_city_five,dict_city_six]

In [279]:
combined_city_dict = {key:value for x in list_world_cities for key,value in x.items() }

In [280]:
combined_city_dict

{'ljudinovo': 'lyudinovo',
 'lubjany': 'lubyany',
 'lobanovo': 'lobanovo',
 'linda': 'lindi',
 'leskolovo': 'leskolovo',
 'koesjva': 'kushva',
 'koerganinsk': 'kurganinsk',
 'krasn.tkach': 'krasnyy tkach',
 "koshekhabl'": "koshekhabl'",
 'korjazhma': 'koryazhma',
 'korobicyno': 'korobitsyno',
 'kondrova': 'kondrovo',
 'kokina': 'kokino',
 'klimov': 'klimovo',
 'kirovskij': 'balpyk bi',
 'kiricy': 'kiritsy',
 'kevsala': 'kevsala',
 'karkazskiy': 'kavkazskiy',
 'karata': 'karata',
 'izhoski': 'iznoski',
 "iban'nkoront": 'ivangorod',
 'isembai': 'ishimbay',
 'inozemcevo': 'inozemtsevo',
 'im.vorovskogo': 'imeni vorovskogo',
 'igra': 'igra',
 'goebacha': 'gubakha',
 'germenchik': 'germenchik',
 'ehtoka': 'etoka',
 'ehlektragorsk': 'elektrogorsk',
 'damadzedava': 'domodedovo',
 'dobroe': 'dobre',
 'dmitrievka': 'dmytrivka',
 'chusavoj': 'chusovoy',
 'chernukha': 'chernukha',
 'chernomorskij': 'chernomorskiy',
 'bazorkino': 'chermen',
 'batalpashinsk': 'cherkessk',
 'cajkovski': 'chaykovskiy

In [281]:
# adding edge cases 
dict_city_name['frankfurt'] = 'frankfurt am main'


In [282]:
df_user_data_cleaned['user_city'] = np.nan

In [283]:
df_user_data_cleaned.sample(20)

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2,user_country,user_city
13129,127828611@n02,True,<NA>,tbwlc photography,<NA>,de delà à delà,2008-01-01 18:12:14,2015-01-10 18:34:17,4366,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,de delà à delà,0,0,NaN,NaN
6328,87857101@n00,False,kdlb,kdlb,kristine beeson,"vancouver, usa",2005-04-05 08:51:21,2005-04-05 15:51:21,8087,pacific time (us & canada); tijuana,-08:00,pst8pdt,vancouver,usa,0,united states of america,NaN
5060,191596708@n07,False,androidofmyeye,d.e.s.i.d.e.r.a.t.a,you're the android of my eye,suburbia,2021-01-01 17:44:28,2021-01-02 01:44:43,852,pacific time (us & canada); tijuana,-08:00,pst8pdt,suburbia,0,0,NaN,NaN
2378,31517223@n00,True,harrywells,greg riekens,greg riekens,"shakopee, mn, usa",2005-06-18 19:18:24,2005-06-29 01:20:10,2395,central time (us & canada),-06:00,cst6cdt,shakopee,mn,usa,united states of america,NaN
2215,53951073@n03,True,italia_city,paolo dell'angelo (journeytoitaly),paolo dell'angelo (journeytoitaly),"gorizia, italia",2008-08-02 22:36:19,2010-09-15 00:07:11,2071,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,gorizia,italia,0,italy,NaN
11897,73267665@n00,False,pigstie,lena_mollergren,lena möllergren,"bankeryd, jönköping, sweden",1918-06-08 12:54:18,2006-10-21 19:19:49,1518,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,bankeryd,jönköping,sweden,sweden,NaN
3410,77849983@n00,True,<NA>,joe with a camera,joe botting,"llandrindod, uk",NaT,2007-02-26 09:50:04,3563,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,llandrindod,uk,0,united kingdom,NaN
439,32760034@n08,True,franck-chilli,frankartculinary,frank müller,"hamburg, germany",2000-10-17 10:12:41,2008-11-23 07:21:34,5856,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,hamburg,germany,0,germany,NaN
6559,25934037@n05,False,emarinuk,emarinuk,ed marin,"dundee, scotland",2006-10-08 11:19:12,2008-04-24 00:13:58,1581,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,dundee,scotland,0,scotland,NaN
3584,197459231@n07,True,ashleyspice,spice_force,ashley spice,"winchester, va, united states",2010-08-11 03:46:39,2023-05-17 03:20:20,317,pacific time (us & canada); tijuana,-08:00,pst8pdt,winchester,va,united states,united states of america,NaN


In [284]:
df_user_data_cleaned.loc[12316]

user_id                                                           37357556@n00
user_ispro                                                                True
user_path_alias                                                      andreaffm
user_username                                                        andreaffm
user_realname                                                    andrea diener
user_location                                               frankfurt, germany
user_photos_firstdatetaken                                 2004-03-14 01:13:51
user_photos_firstdate                                      2005-10-18 18:56:31
user_photos_count                                                         5309
user_timezone_label           amsterdam, berlin, bern, rome, stockholm, vienna
user_timezone_offset                                                    +01:00
user_timezone_timezone_id                                     europe/amsterdam
0                                                   

In [285]:
# for index, row in df_user_data_cleaned.iterrows():
#     x = row['0']
#     if pd.isna(row['user_city']):
#         if x in combined_city_dict:
#             df_user_data_cleaned.at[index, 'user_city']= combined_city_dict[x]

In [286]:
combined_city_dict['frankfurt']

'frankfort'

In [287]:
dict_city_name['frankfurt']

'frankfurt am main'

## frankfurt causes trouble bc there is an alternative name of frankfort in us
so I decided to run the real name dict prior

In [288]:
for index, row in df_user_data_cleaned.iterrows():
    x = row['0']
    if pd.isna(row['user_city']):
        if x in dict_city_name:
            df_user_data_cleaned.at[index, 'user_city']= dict_city_name[x]

In [289]:
for index, row in df_user_data_cleaned.iterrows():
    x = row['1']
    if pd.isna(row['user_city']):
        if x in dict_city_name:
            df_user_data_cleaned.at[index, 'user_city']= dict_city_name[x]

In [290]:
for index, row in df_user_data_cleaned.iterrows():
    x = row['2']
    if pd.isna(row['user_city']):
        if x in dict_city_name:
            df_user_data_cleaned.at[index, 'user_city']= dict_city_name[x]

In [291]:
df_user_data_cleaned.head(20)

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2,user_country,user_city
0,26745338@n06,False,danisonksen,dani.sonksen,daniela sönksen,"melbourne, australia",2005-05-09 10:56:54,2008-06-05 20:03:04,850,brasilia,-03:00,america/sao_paulo,melbourne,australia,0,australia,melbourne
1,31331368@n00,True,lonelyradio,lonely radio,andrew j. cosgriff,"melbourne, australia",2000-02-01 00:00:00,2004-11-07 06:59:09,13381,"canberra, melbourne, sydney",+10:00,australia/canberra,melbourne,australia,0,australia,melbourne
2,122687277@n03,True,philipmallis,philip.mallis,philip mallis,melbourne,2007-09-21 04:37:37,2014-04-13 12:48:29,10793,<NA>,<NA>,<NA>,melbourne,0,0,NaN,melbourne
3,45008553@n04,False,reaksmeyyean,reaksmey yean – george,reaksmey yean – george,"phnom penh, kingdom of cambodia",2005-11-04 20:32:27,2009-11-27 02:43:31,236,"bangkok, hanoi, jakarta",+07:00,asia/bangkok,phnom penh,kingdom of cambodia,0,cambodia,phnom penh
4,158117692@n02,True,<NA>,julie mclennan,julie mclennan,"melbourne, australia",2018-01-08 22:23:51,2018-01-09 06:23:51,2533,<NA>,<NA>,<NA>,melbourne,australia,0,australia,melbourne
5,169491036@n06,False,<NA>,sanzidrahmankhantamim,sanzid rahman khan tamim,"barishal, bangladesh",2015-03-24 23:04:05,2019-04-06 13:35:11,407,<NA>,<NA>,<NA>,barishal,bangladesh,0,bangladesh,barishal
6,153546415@n07,True,free_aza_bird,free_aza_bird,karl baker,"melbourne, australia",2015-06-22 08:50:59,2017-04-03 01:24:16,2674,"canberra, melbourne, sydney",+10:00,australia/canberra,melbourne,australia,0,australia,melbourne
7,78379788@n06,True,<NA>,rob lee photography,rob lee,australia,1985-01-01 00:00:00,2014-08-20 06:35:22,2867,"canberra, melbourne, sydney",+10:00,australia/canberra,australia,0,0,australia,NaN
8,115521748@n08,False,<NA>,footeefok,tee fok foo,"singapore, singapore",2008-08-24 12:38:45,2014-01-25 15:18:44,20678,"kuala lumpur, singapore",+08:00,asia/kuala_lumpur,singapore,singapore,0,singapore,singapore
9,44340545@n05,True,gmpolice1,greater manchester police,greater manchester police,"manchester, greater manchester",1840-01-01 00:00:00,2009-11-05 13:53:07,27764,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,manchester,greater manchester,0,NaN,manchester


In [292]:
df_user_data_cleaned.user_city.isna().sum()

3453

In [293]:
for index, row in df_user_data_cleaned.iterrows():
    x = row['0']
    if pd.isna(row['user_city']):
        if x in combined_city_dict:
            df_user_data_cleaned.at[index, 'user_city']= combined_city_dict[x]

In [294]:
for index, row in df_user_data_cleaned.iterrows():
    x = row['1']
    if pd.isna(row['user_city']):
        if x in combined_city_dict:
            df_user_data_cleaned.at[index, 'user_city']= combined_city_dict[x]

In [295]:
for index, row in df_user_data_cleaned.iterrows():
    x = row['2']
    if pd.isna(row['user_city']):
        if x in combined_city_dict:
            df_user_data_cleaned.at[index, 'user_city']= combined_city_dict[x]

In [296]:
df_user_data_cleaned.sample(15)

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2,user_country,user_city
11343,66749627@n00,False,<NA>,spudmunkey_dot_com,stephen wittmaak,"st. petersburg, fl, united states",2008-01-31 21:48:00,2008-10-01 02:36:18,2136,eastern time (us & canada),-05:00,est5edt,st. petersburg,fl,united states,united states of america,st. petersburg
2258,66014106@n00,True,agrinberg,agrinberg,alan grinberg,"pacifica, california, united states",1973-08-01 00:00:00,2007-02-09 07:34:28,3449,pacific time (us & canada); tijuana,-08:00,pst8pdt,pacifica,california,united states,united states of america,pacifica
2929,51794673@n00,True,vruba,vruba,charlie loyd,"94608, usa",2001-01-01 00:00:00,2006-09-14 21:39:03,2463,pacific time (us & canada); tijuana,-08:00,pst8pdt,94608,usa,0,united states of america,usa
8372,12203749@n05,True,isagalaev,ivan sagalaev,ivan sagalaev,"sammamish, wa, usa",2008-09-13 16:51:22,2013-05-21 05:55:55,360,pacific time (us & canada); tijuana,-08:00,pst8pdt,sammamish,wa,usa,united states of america,sammamish
735,106705450@n03,True,<NA>,caulker,tanya kostina,"greater london, uk",2002-07-01 00:00:00,2013-10-26 15:30:08,4584,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,greater london,uk,0,united kingdom,uk
9074,58167807@n00,True,elmsn,_nur,núria,"el masnou, maresme",NaT,2005-12-10 18:12:56,5417,"brussels, copenhagen, madrid, paris",+01:00,europe/brussels,el masnou,maresme,0,NaN,el masnou
3127,52563475@n00,True,mikeandlouise,shaggy.dog,mike price,uk,2000-01-01 00:08:38,2008-10-06 20:57:05,13697,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,uk,0,0,united kingdom,uk
7594,49879525@n08,True,hoteidoc,hoteidoc,peter brown,"greybulll, wy, us",2009-05-10 21:34:51,2011-01-14 00:08:28,718,eastern time (us & canada),-05:00,est5edt,greybulll,wy,us,united states of america,us
5267,28045633@n00,True,h_duncan,howard j duncan,howard duncan,"victoria bc, canada",2001-02-01 17:12:01,2007-07-02 02:26:17,4784,pacific time (us & canada); tijuana,-08:00,pst8pdt,victoria bc,canada,0,canada,canada
8497,132136419@n04,True,dscjudo,dsc-judo,dsc wanne-eickel - judo e.v.,"wanne-eickel, deutschland",NaT,2015-08-19 16:20:23,59789,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,wanne-eickel,deutschland,0,germany,NaN


In [297]:
df_user_data_cleaned.tail(20)

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2,user_country,user_city
14449,46073170@n05,True,flow_of_pictures,kptnflow,florian fievet,"montréal, canada",1990-01-01 00:00:00,2010-01-02 16:13:29,5002,eastern time (us & canada),-05:00,est5edt,montréal,canada,0,canada,montreal
14450,191523917@n07,True,huub-janssen,huub.janssen,huub janssen,"venlo-blerick, nederland",2020-11-24 06:15:08,2020-12-24 10:37:27,3336,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,venlo-blerick,nederland,0,netherlands,nederland
14451,44372192@n00,True,matisok,matisok,mathias willerup,"copenhagen, denmark",1983-07-01 00:00:00,2000-05-22 21:14:44,4048,"brussels, copenhagen, madrid, paris",+01:00,europe/brussels,copenhagen,denmark,0,denmark,copenhagen
14452,18839565@n00,False,daniel_pettinger,daniel pettinger,daniel pettinger,"linz, austria",2004-03-29 19:48:16,2006-01-15 16:31:41,10564,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,linz,austria,0,austria,linz
14453,68574290@n00,False,julien-falgas,webcomics.fr,<NA>,france,2007-06-03 22:24:11,2007-06-03 20:24:11,53,"brussels, copenhagen, madrid, paris",+01:00,europe/brussels,france,0,0,france,NaN
14454,24642077@n02,False,efrensl,efrén sl,efrén sánchez,españa,2010-03-13 17:38:41,2010-03-25 22:37:12,347,"brussels, copenhagen, madrid, paris",+01:00,europe/brussels,españa,0,0,spain,espana
14455,65059720@n06,False,<NA>,marc sullivan,marc sullivan,"hamburg, deutschland",2013-06-26 17:54:37,2019-03-22 21:33:41,146,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,hamburg,deutschland,0,germany,hamburg
14456,95482784@n00,False,robaer,robert fritz,robert fritz,austria,2005-10-02 14:14:12,2006-05-08 11:02:33,439,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,austria,0,0,austria,NaN
14457,191808253@n06,False,champland,champland,au fil des routes et des chemins de.... ...,"marne, france",1981-01-01 00:00:00,2021-01-18 22:20:56,863,"brussels, copenhagen, madrid, paris",+01:00,europe/brussels,marne,france,0,france,NaN
14458,51485383@n02,False,sergioloca,agoralex,sergio locatelli,"casorate primo (pv), italia",2009-05-01 17:50:43,2010-07-03 22:00:37,939,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,casorate primo (pv),italia,0,italy,NaN


In [298]:
## check edge cases

In [299]:
df_user_data_cleaned.loc[9958]

user_id                                                 34462855@n03
user_ispro                                                      True
user_path_alias                                        andyhemingway
user_username                                               andy_ahg
user_realname                                         andy hemingway
user_location                                                     uk
user_photos_firstdatetaken                       2003-02-10 22:26:37
user_photos_firstdate                            2009-01-17 22:16:39
user_photos_count                                               3698
user_timezone_label           gmt: dublin, edinburgh, lisbon, london
user_timezone_offset                                          +00:00
user_timezone_timezone_id                              europe/london
0                                                                 uk
1                                                                  0
2                                 

In [300]:
df_user_data_cleaned.loc[5935]

user_id                                              24112737@n05
user_ispro                                                  False
user_path_alias                                     klaasverpoest
user_username                                       klaasverpoest
user_realname                                      klaas verpoest
user_location                                             belgium
user_photos_firstdatetaken                    2007-02-08 19:59:45
user_photos_firstdate                         2008-02-25 16:39:08
user_photos_count                                             453
user_timezone_label           brussels, copenhagen, madrid, paris
user_timezone_offset                                       +01:00
user_timezone_timezone_id                         europe/brussels
0                                                         belgium
1                                                               0
2                                                               0
user_count

In [301]:
df_user_data_cleaned.loc[8]

user_id                                 115521748@n08
user_ispro                                      False
user_path_alias                                  <NA>
user_username                               footeefok
user_realname                             tee fok foo
user_location                    singapore, singapore
user_photos_firstdatetaken        2008-08-24 12:38:45
user_photos_firstdate             2014-01-25 15:18:44
user_photos_count                               20678
user_timezone_label           kuala lumpur, singapore
user_timezone_offset                           +08:00
user_timezone_timezone_id           asia/kuala_lumpur
0                                           singapore
1                                           singapore
2                                                   0
user_country                                singapore
user_city                                   singapore
Name: 8, dtype: object

In [302]:
df_user_data_cleaned.shape

(14172, 17)

In [303]:
df_user_data_cleaned.user_city.isna().sum()

2827

In [304]:
df_user_data_cleaned.user_country.isna().sum()

2398

In [305]:
df_world_cities.sample(7)

,geoname_id,name,ascii_name,alternate_names,feature_class,feature_code,country_code,country_name_en,country_code_2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,digital_elevation_model,timezone,modification_date,label_en,coordinates
32653,1688216,Santa Cruz,Santa Cruz,"sheng ke lu si qu,聖克魯斯區",P,PPLX,PH,Philippines,NaN,NCR,133900000,133903000,NaN,126735,NaN,15,Asia/Manila,2022-08-09,Philippines,"14.6, 120.98333"
5790,1680623,Tupang,Tupang,Tupang,P,PPL,PH,Philippines,NaN,02,14,021502000,NaN,2213,NaN,24,Asia/Manila,2017-12-13,Philippines,"17.90483, 121.64099"
42507,3117504,Manzanares de Rioja,Manzanares de Rioja,Manzanares de Rioja,P,PPLA3,ES,Spain,NaN,27,LO,26094,NaN,99,NaN,803,Europe/Madrid,2012-03-04,Spain,"42.3961, -2.8959"
1420,694558,Semypolky,Semypolky,"Semipolki,Semypolky,Семиполки",P,PPL,UA,Ukraine,NaN,13,3206,NaN,NaN,2939,NaN,116,Europe/Kyiv,2022-10-27,Ukraine,"50.72627, 30.93441"
12305,12420928,Huangdi,Huangdi,"huang de,荒地",P,PPLA4,CN,China,NaN,13,6531,NaN,NaN,34658,NaN,1174,Asia/Urumqi,2022-02-05,China,"38.80898, 77.44861"
116385,3626809,Sinamaica,Sinamaica,Sinamaica,P,PPLA2,VE,"Venezuela, Bolivarian Rep. of",NaN,23,2315,NaN,NaN,0,NaN,0,America/Caracas,2014-09-28,"Venezuela, Bolivarian Rep. of","11.08656, -71.85595"
57577,2809214,Wiesensteig,Wiesensteig,"Vizenshtajg,fyznshtaygh,wei sen si tai xi,Визе...",P,PPL,DE,Germany,NaN,01,081,08117,08117058,2199,NaN,590,Europe/Berlin,2013-02-19,Germany,"48.56127, 9.6254"


In [306]:
world_city_columns_keep = ['ascii_name','country_name_en','country_code','population','timezone','coordinates']

In [307]:
df_world_cities = df_world_cities.loc[:,world_city_columns_keep]

In [308]:
# add prefix except ascii_name
df_world_cities = df_world_cities.add_prefix('city_')


In [309]:
#df_world_cities['user_city'] = df_world_cities['city_ascii_name']

In [310]:
df_world_cities.sample(5)

,city_ascii_name,city_country_name_en,city_country_code,city_population,city_timezone,city_coordinates
102427,Icod de los Vinos,Spain,ES,24024,Atlantic/Canary,"28.37241, -16.71188"
14786,Tocumbo,Mexico,MX,1799,America/Mexico_City,"19.70177, -102.52323"
1814,Tigliole,Italy,IT,154,Europe/Rome,"44.88625, 8.07663"
90077,Gongcheng,China,CN,0,Asia/Shanghai,"24.83429, 110.82423"
117483,Malaunay,France,FR,5943,Europe/Paris,"49.5271, 1.04292"


In [311]:
#df_world_cities[df_world_cities.city_country_name_en=="Korea, Republic of"]

In [312]:
# df_world_countries.head()
# df_world_countries[df_world_countries.alpha2=='cz']

In [313]:
#df_world_countries['Czech Republic']='Czechia'

In [314]:
#df_user_data_cleaned_city = df_user_data_cleaned.merge(df_world_cities, left_on='user_city',right_on='city_ascii_name',how='left')

In [315]:
#df_user_data_cleaned_country=df_user_data_cleaned.merge(df_world_cities, left_on='user_country',right_on='city_country_name_en',how='left')

In [316]:
#df_user_data_cleaned_country.head(20)

In [317]:
df_world_cities.city_ascii_name = df_world_cities.city_ascii_name.str.lower()

In [318]:
df_world_cities.city_country_name_en = df_world_cities.city_country_name_en.str.lower()
df_user_data_cleaned.user_country = df_user_data_cleaned.user_country.str.lower()
df_world_cities.city_timezone = df_world_cities.city_timezone.str.lower()

In [319]:
df_world_cities = df_world_cities.apply(lambda x: x.astype('string').str.strip() if x.dtype == "object" else x) 

In [320]:
df_world_cities.head()

,city_ascii_name,city_country_name_en,city_country_code,city_population,city_timezone,city_coordinates
0,lyudinovo,russian federation,RU,41392,europe/moscow,"53.86639, 34.44778"
1,lubyany,russian federation,RU,1899,europe/moscow,"56.0378, 51.40001"
2,lobanovo,russian federation,RU,3563,asia/yekaterinburg,"57.8595, 56.3026"
3,linda,russian federation,RU,5489,europe/moscow,"56.61571, 44.09544"
4,leskolovo,russian federation,RU,4039,europe/moscow,"60.26341, 30.45462"


In [321]:
df_world_cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141121 entries, 0 to 141120
Data columns (total 6 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   city_ascii_name       141120 non-null  string
 1   city_country_name_en  140953 non-null  string
 2   city_country_code     141076 non-null  string
 3   city_population       141121 non-null  int64 
 4   city_timezone         141121 non-null  string
 5   city_coordinates      141121 non-null  string
dtypes: int64(1), string(5)
memory usage: 6.5 MB


In [322]:
# split coordinates and merge to df world cities
df_c_split = df_world_cities.city_coordinates.str.split(', ', expand=True)
df_c_split.columns = df_c_split.columns.astype('string')
df_c_split = df_c_split.rename(columns={'0':'city_lat','1':'city_long'})

In [323]:
df_c_split.head()

,city_lat,city_long
0,53.86639,34.44778
1,56.0378,51.40001
2,57.8595,56.3026
3,56.61571,44.09544
4,60.26341,30.45462


In [324]:
df_world_cities = df_world_cities.merge(df_c_split, left_index=True,right_index=True,how='left')

In [325]:
df_world_cities.head()

,city_ascii_name,city_country_name_en,city_country_code,city_population,city_timezone,city_coordinates,city_lat,city_long
0,lyudinovo,russian federation,RU,41392,europe/moscow,"53.86639, 34.44778",53.86639,34.44778
1,lubyany,russian federation,RU,1899,europe/moscow,"56.0378, 51.40001",56.0378,51.40001
2,lobanovo,russian federation,RU,3563,asia/yekaterinburg,"57.8595, 56.3026",57.8595,56.3026
3,linda,russian federation,RU,5489,europe/moscow,"56.61571, 44.09544",56.61571,44.09544
4,leskolovo,russian federation,RU,4039,europe/moscow,"60.26341, 30.45462",60.26341,30.45462


In [326]:
df_user_data_cleaned_final=df_user_data_cleaned.merge(df_world_cities, left_on=['user_country','user_city'],right_on=['city_country_name_en','city_ascii_name'],how='left')

In [327]:
df_user_data_cleaned_final.head(20)

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,...,user_country,user_city,city_ascii_name,city_country_name_en,city_country_code,city_population,city_timezone,city_coordinates,city_lat,city_long
0,26745338@n06,False,danisonksen,dani.sonksen,daniela sönksen,"melbourne, australia",2005-05-09 10:56:54,2008-06-05 20:03:04,850,brasilia,...,australia,melbourne,melbourne,australia,AU,4917750.0,australia/melbourne,"-37.814, 144.96332",-37.814,144.96332
1,31331368@n00,True,lonelyradio,lonely radio,andrew j. cosgriff,"melbourne, australia",2000-02-01 00:00:00,2004-11-07 06:59:09,13381,"canberra, melbourne, sydney",...,australia,melbourne,melbourne,australia,AU,4917750.0,australia/melbourne,"-37.814, 144.96332",-37.814,144.96332
2,122687277@n03,True,philipmallis,philip.mallis,philip mallis,melbourne,2007-09-21 04:37:37,2014-04-13 12:48:29,10793,<NA>,...,NaN,melbourne,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>
3,45008553@n04,False,reaksmeyyean,reaksmey yean – george,reaksmey yean – george,"phnom penh, kingdom of cambodia",2005-11-04 20:32:27,2009-11-27 02:43:31,236,"bangkok, hanoi, jakarta",...,cambodia,phnom penh,phnom penh,cambodia,KH,1573544.0,asia/phnom_penh,"11.56245, 104.91601",11.56245,104.91601
4,158117692@n02,True,<NA>,julie mclennan,julie mclennan,"melbourne, australia",2018-01-08 22:23:51,2018-01-09 06:23:51,2533,<NA>,...,australia,melbourne,melbourne,australia,AU,4917750.0,australia/melbourne,"-37.814, 144.96332",-37.814,144.96332
5,169491036@n06,False,<NA>,sanzidrahmankhantamim,sanzid rahman khan tamim,"barishal, bangladesh",2015-03-24 23:04:05,2019-04-06 13:35:11,407,<NA>,...,bangladesh,barishal,barishal,bangladesh,BD,202242.0,asia/dhaka,"22.70497, 90.37013",22.70497,90.37013
6,153546415@n07,True,free_aza_bird,free_aza_bird,karl baker,"melbourne, australia",2015-06-22 08:50:59,2017-04-03 01:24:16,2674,"canberra, melbourne, sydney",...,australia,melbourne,melbourne,australia,AU,4917750.0,australia/melbourne,"-37.814, 144.96332",-37.814,144.96332
7,78379788@n06,True,<NA>,rob lee photography,rob lee,australia,1985-01-01 00:00:00,2014-08-20 06:35:22,2867,"canberra, melbourne, sydney",...,australia,NaN,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>
8,115521748@n08,False,<NA>,footeefok,tee fok foo,"singapore, singapore",2008-08-24 12:38:45,2014-01-25 15:18:44,20678,"kuala lumpur, singapore",...,singapore,singapore,singapore,singapore,SG,5638700.0,asia/singapore,"1.28967, 103.85007",1.28967,103.85007
9,44340545@n05,True,gmpolice1,greater manchester police,greater manchester police,"manchester, greater manchester",1840-01-01 00:00:00,2009-11-05 13:53:07,27764,"gmt: dublin, edinburgh, lisbon, london",...,NaN,manchester,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>


In [ ]:
df_user_data_cleaned_final.loc[10]

In [ ]:
df_user_data_cleaned_final[df_user_data_cleaned_final.user_country=='Czech Republic']

In [ ]:
df_user_data_cleaned_final.shape

In [ ]:
df_user_data_cleaned_final.city_coordinates.isna().sum()

In [ ]:
#df_user_data_cleaned_city.shape

In [ ]:
#df_user_data_cleaned_country.shape

In [ ]:
df_user_data_cleaned.sample(15)

In [ ]:
# compare countries
#df_user_data_cleaned['equal_country'] = df_user_data_cleaned['user_country'].eq(df_user_data_cleaned['city_country_name_en'])

In [ ]:
df_user_data_cleaned_final.head(30)

In [ ]:
# keep false + NaN
#df_user_data_cleaned[df_user_data_cleaned['equal_country']== True]

In [ ]:
df_user_data_cleaned_final.city_coordinates.count()

In [ ]:
df_user_data_cleaned_final[df_user_data_cleaned_final[user_country]

In [ ]:
df_user_data_cleaned_final.shape


In [ ]:
df_user_data_cleaned_final.info()